In [32]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [34]:
from tuna import TunaCorpus
from glob import glob


CORPUS_FILES = "../TUNA/corpus/*/*/*.xml"

In [39]:
corpus = TunaCorpus(glob(CORPUS_FILES))
corpus

<tuna.TunaCorpus instance at 0x7fb8fc6f3488>

In [40]:
trials = list(corpus.iter_trials())

In [41]:
trials[0].entities[0].attributes_as_dict()

{'colour': 'red',
 'orientation': 'back',
 'size': 'small',
 'type': 'desk',
 'x-dimension': '5',
 'y-dimension': '3'}

In [45]:
import itertools
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer

def featurize(description, entity):
    return Counter(['%s+%s:%s' % (unigram, key, value)
                    for unigram in description.unigrams()
                    for key, value in entity.attributes_as_dict().iteritems()])


features, target = zip(*[(featurize(trial.description, e), e.is_target())
                         for trial in corpus.iter_trials()
                         for e in trial.entities])

learner = LogisticRegression(solver='lbfgs')
vec = DictVectorizer()
features = vec.fit_transform(features)

In [46]:
features.shape

(17460, 15461)

In [54]:
vec.get_feature_names()[1100:1110]

['among+hasTie:0',
 'among+hasTie:1',
 'among+orientation:front',
 'among+orientation:left',
 'among+orientation:right',
 'among+type:person',
 'among+x-dimension:1',
 'among+x-dimension:2',
 'among+x-dimension:3',
 'among+x-dimension:4']

In [60]:
from sklearn.cross_validation import cross_val_predict

predicted = cross_val_predict(learner, features, target, cv=10, verbose=4)

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    5.7s finished


NameError: name 'metrics' is not defined

In [61]:
from sklearn import metrics
print metrics.precision_score(target, predicted)
print metrics.recall_score(target, predicted)
print metrics.f1_score(target, predicted)

0.216494845361
1.0
0.35593220339


In [62]:
Counter(predicted)

Counter({True: 17460})

In [63]:
Counter(target)

Counter({False: 13680, True: 3780})